In [1]:
##############
#K. David Roell CFPB 7/12/16
#Compiles a report of all quality edit failures for HMDA LAR data
#
##############
import json
import os
import pandas as pd
import psycopg2
#from macro_sql import Q076 
with open('quality_sql.json') as f: #FIXME change to quality_sql.json and refactor the rest of code
    edit_sql = json.load(f)
    
#conn = psycopg2.connect("dbname=hmdamaster user=roellk") #connect and return connection
#cur = conn.cursor()#instantiate cursor object to use in SQL queries
#parameter format for local use #consider changing hmdamaster to roellk if db changes cause a fail
params = {
'dbname':'hmdamaster',
'user':'roellk',
'password':'',
'host':'localhost',

}

#connect_string = "dbname=%s user=%s host=%s password =%s" %(dbname, user, host, password) #set a string for connection to SQL
try:
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    print("i'm connected")

except psycopg2.Error as e: #if database connection results in an error print the following
    print("I am unable to connect to the database: ", e)


lar_tables = ['hmdalar2014', 'hmdalar2013', 'hmdalar2012', 'hmdalar2011', 'hmdalar2010', 'hmdalar2009'
             ,'hmdalar2008', 'hmdalar2007', 'hmdalar2006', 'hmdalar2005', 'hmdalar2004', 'hmdalar2003'
             ,'hmdalar2002', 'hmdalar2001', 'hmdalar2000', 'hmdalar1999', 'hmdalar1998', 'hmdalar1997'
             ,'hmdalar1996', 'hmdalar1995', 'hmdalar1994', 'hmdalar1993']



i'm connected


In [2]:
def edit_text(table='hmdalar2014', edit_sql=edit_sql[0]['text'], edit_name=edit_sql[0]['name'], add_fields=[]):
    base_sql = """SELECT year, agency, COUNT(sequence) AS {edit_name} """.format(edit_name=edit_name)
    table_sql = "\nFROM {table}\n".format(table=table) 
    group_sql = """ GROUP BY year, agency"""
    for field in add_fields:
        base_sql = base_sql + ',' + field
        group_sql = group_sql + ', '+ field
    return_sql = base_sql + table_sql + edit_sql + group_sql + ';'
    print(return_sql)
    return return_sql 

In [4]:

fields=['RID']
for table in lar_tables:
    first = True
    for edit in edit_sql:
        name, text = edit['name'], edit['text']
        print(name)
        print(table)
        cur.execute(edit_text(table=table, edit_sql=text, edit_name=name, add_fields=fields))        
        data_df = pd.DataFrame(cur.fetchall())

        col_names = [desc[0] for desc in cur.description]
        if len(data_df.columns) == len(col_names):
            print('setting column names')
            data_df.columns = col_names

        if first == True and len(data_df.columns) > 0:
            quality_df = data_df.copy()
            first = False
                                 
        elif first == False and len(data_df.columns) > 0:
            print('merging dataframes')
            quality_df=quality_df.merge(data_df, how='outer', on=['agency', 'year', 'rid'])
    
        else:
            print('no data returned from query')
        
        print(quality_df.head())

    quality_df['arid'] = quality_df.agency.map(str) + quality_df.rid.map(str)
    quality_df.drop('rid', axis=1, inplace=True)
    path = 'quality_csvs/'
    if not os.path.exists(path):
        os.makedirs(path)
    quality_df.to_csv(path + 'quality_edits' + table[-4:] +'.csv', index=False)

Q035
hmdalar2014
SELECT year, agency, COUNT(sequence) AS Q035 ,RID
FROM hmdalar2014
WHERE loan_type != '1' AND purchaser IN  ('1','3') GROUP BY year, agency, RID;
setting column names
   year agency  q035         rid
0  2014      3     3  0000026967
1  2014      5     3  0000062542
2  2014      1     2  0000014062
3  2014      2     2  0000456157
4  2014      2     2  0000529958
Q001
hmdalar2014
SELECT year, agency, COUNT(sequence) AS Q001 ,RID
FROM hmdalar2014
WHERE amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' AND amount::int >= 1000 AND (amount::int / income::int) >= 5 GROUP BY year, agency, RID;
setting column names
merging dataframes
   year agency  q035         rid  q001
0  2014      3   3.0  0000026967   NaN
1  2014      5   3.0  0000062542   NaN
2  2014      1   2.0  0000014062   NaN
3  2014      2   2.0  0000456157   NaN
4  2014      2   2.0  0000529958   NaN
Q002
hmdalar2014
SELECT year, agency, COUNT(sequence) AS Q002 ,RID
FROM hmdalar2014
WHERE property_type = '1' AND

ProgrammingError: column "property_type" does not exist
LINE 3: WHERE property_type = '1' AND income NOT ILIKE '%NA%' AND am...
              ^


In [5]:
print('test')

test
